In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# files = os.path.join(data, "simulation_*_output.csv")
# files = glob.glob(files)
# df = pd.concat(map(pd.read_csv, files), ignore_index=True)
# df = df.replace(-1, np.nan)
# df
# values  = []
# for i in range(0,5):
#     df_mean = df[i::5].mean()
#     values.append(df_mean.tolist())
# len(values[0])
# new = []
# for i in range(len(values)):
#     dummy = []
#     if i == 0 or i ==3:
#         for j in values[i]:
#             j -= 2500
#             dummy.append(j)
#     if i == 1 or i ==2:
#         for j in values[i]:
#             j -= 4000
#             dummy.append(j)
#     if i == 4:
#         for j in values[i]:
#             j += 2000
#             dummy.append(j)
#     new.append(dummy)
# new
# sfile = pd.read_csv("../input/first-arrivals-ml-in-physics/sampleSubmission.csv")
# sfile["Predicted"] = matrix[0]
# sfile
# sfile.to_csv("test.csv",index=False)

In [62]:
checker = pd.read_csv('../input/first-arrivals-ml-in-physics/data/train/simulation_1_output.csv')
checker

In [5]:
import os
import tensorflow as tf
import numpy as np
import pathlib
from matplotlib import pyplot as plt
import pandas as pd
import glob
from scipy.fft import fft, ifft
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [6]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.eperimental.set_memory_growth(gpu, True)

In [7]:
data = '../input/first-arrivals-ml-in-physics/data/train'
data_dir = os.listdir(data)

In [8]:
# import re
# def numericalSort(value):
#     numbers = re.compile(r'(\d+)')
#     parts = numbers.split(value)
#     parts[1::2] = map(int, parts[1::2])
#     return parts
# fitsf=sorted(glob.glob( os.path.join(data, "simulation_*/receiver_0.csv")), key=numericalSort)
# fitsf

In [9]:
files = os.path.join(data, "simulation_*/receiver_0.csv")
files = glob.glob(files)
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
df

In [10]:
X_list =df[["Timestep","X"]]
Y_list =df[["Timestep","Y"]]
Z_list =df[["Timestep","Z"]]

In [11]:
Xs = X_list[:30000].X
Ys = Y_list[:30000].Y
Zs = Z_list[:30000].Z


In [109]:
from sklearn import preprocessing

In [125]:
# X =X_list[17796:17996]
# Y =Y_list[17796:17996]
# Z =Z_list[17796:17996]
# X= X.values.tolist()
X_list[17856:17946].plot(x='Timestep', y='X')
Y_list[17856:17946].plot(x='Timestep', y='Y')
Z_list[17856:17946].plot(x='Timestep', y='Z')
# # Y = Y_list[1100:1200]
# # Z= Z_list[1100:1200]
# # X = pd.DataFrame(X)
# t= np.linspace(0, 98, 98)
# # y = pd.DataFrame([i for i in range(1000)])
# # X = pd.concat([X, y], axis = 1, )
# # X.columns = ["time","coords"]
# # X
# norm = [float(i)/max(X) for i in X]
# norm

In [119]:
grad = []
grad_2 = []
for i in range(0, len(X)-1):
    p = norm[i+1] - norm[i]
    grad.append(p)
len(grad)
for i in range(0, len(grad)-1):
    k = grad[i+1] - grad[i]
    grad_2.append(k)
len(grad_2)

In [122]:
plt.figure(figsize = (20,20))
plt.plot(t, grad_2)

In [13]:
# def spectrogram_gen(input):
#     spectrogram = tf.signal.stft(input, frame_length=256, frame_step=128)
#     spectrogram = tf.abs(spectrogram)
#     spectrogram = tf.expand_dims(spectrogram, axis=2)
#     return spectrogram

In [100]:
Xspec = get_spectrogram(X)
Yspec = get_spectrogram(Y)
Zspec = get_spectrogram(Z)
print(Xspec)

plt.figure(figsize=(15,5))
plt.imshow(tf.transpose(Xspec)[0])
plt.show()

In [51]:
def get_spectrogram(waveform):
  input_len =  20000
  waveform = waveform[:input_len]
  zero_padding = tf.zeros([20000] - tf.shape(waveform),dtype=tf.float32)
  waveform = tf.cast(waveform, dtype=tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(
      equal_length, frame_length=256, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis=2)
  return spectrogram

In [40]:
def plot_spectrogram(spectrogram, ax):
  if len(spectrogram.shape) > 2:
    assert len(spectrogram.shape) == 3
    spectrogram = np.squeeze(spectrogram, axis=-1)
  log_spec = np.log(spectrogram.T + np.finfo(float).eps)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0,
                  np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)

In [18]:
fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(X.shape[0])
axes[0].plot(timescale, X)
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 1])

plot_spectrogram(spectrogram, axes[1])
axes[1].set_title('Spectrogram')
plt.show()

In [ ]:
Xn = np.array(Xs.X)
Yn = np.array(Ys.Y)
Zn = np.array(Zs.Z)

In [19]:
fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(Ys.shape[0])
axes[0].plot(timescale, Yn)
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 30000])

plot_spectrogram(Yspec, axes[1])
axes[1].set_title('Spectrogram')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(Zs.shape[0])
axes[0].plot(timescale, Zn)
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 30000])

plot_spectrogram(Zspec, axes[1])
axes[1].set_title('Spectrogram')
plt.show()

In [54]:
omega = 1/(1-tf.math.sqrt((tf.math.square(Xspec) + tf.math.square(Yspec) + tf.math.square(Zspec))/3))
# omega =tf.pow(Xspec *Yspec*Zspec, 1.0/3.0)
# omega_w = Xs.X+Ys.Y+Zs.Z
omega_n = np.array(omega)
omega_n.shape

In [20]:
# fig, axes = plt.subplots(2, figsize=(12, 8))
# timescale = np.arange(omega_n.shape[0])
# axes[0].plot(timescale, omega_n)
# axes[0].set_title('Waveform')
# axes[0].set_xlim([0, 30000])

# plot_spectrogram(omega, axes[1])
# axes[1].set_title('Spectrogram')
# plt.show()
# omega[0]
tf.math.reduce_min(omega)

In [60]:
dummy = []
for i in range(24):
    X = get_spectrogram(X_list[30000*i: 30000*(i+1)].X)
    Y = get_spectrogram(Y_list[30000*i: 30000*(i+1)].Y)
    Z = get_spectrogram(Z_list[30000*i: 30000*(i+1)].Z)
    O = 1/(1-tf.math.sqrt((tf.math.square(X) + tf.math.square(Y) + tf.math.square(Z))))
    dummy.append(O)
for i in dummy:
    fig, _ = plt.subplots(1, figsize=(5,2))
    plot_spectrogram(i, _)
    axes[1].set_title('Spectrogram')
plt.show()
O

In [35]:
# batch_size = 1
# train_ds = O.batch(batch_size)
# val_ds = val_ds.batch(batch_size)
# O = tf.expand_dims(O, axis = 0) 
O = tf.squeeze(O, axis=0)
O

In [23]:
commands_files = [0]
val

In [24]:
num_labels = len(commands_files)
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(233, 129,1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Conv2D(32, (3,3), activation ='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
tf.keras.layers.Dense(2, activation="sigmoid")
model.summary()

In [26]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [33]:
EPOCHS = 5
history = model.fit(
    O,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)